Last time I used the titanic dataset I used AutoML and got in the 35 percentile. This time I will take what I've been learning in the ML DP 100 and try transforming the data and running logistic regression with AutoML hyperdrive tuninign to find a good solution. 

In [1]:
import pandas as pd

training_set = pd.read_csv('train.csv')

In [2]:
training_set.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


Lets first try using the AutoML.

In [3]:
from azureml.core import Workspace

ws=Workspace.get(name ='ms-learn-ml', subscription_id='375be063-544d-4b99-aed9-5a0e16b1f428', resource_group='ms-learn-ml')

In [4]:
import azureml.core
from azureml.core import Dataset

default_ds=ws.get_default_datastore()

#get the training data from our newly uploaded titanic training data
tab_data_set = Dataset.Tabular.from_delimited_files(path = (default_ds, 'titanic-data/train.csv'))
#while I'm at it, I'm going to import the test data that I will run my model on for the competition
tab_test_data_set = Dataset.Tabular.from_delimited_files(path = (default_ds, 'titanic-data/test.csv'))

#if 'titanic data' isn't in ws.dataset we upload it to our datastore and overwrite if needed (this allows for more robust notebook we can run again)
if 'titanic training dataset' not in ws.datasets:
    
    default_ds.upload_files(files = ['./train.csv'],
                        target_path = 'titanic-data/',
                        overwrite = True,
                        show_progress = True
                       )
    
    try:
        tab_data = tab_data_set.register(workspace=ws,
                                         name = 'titanic training dataset',
                                         description = 'titanic training dataset for kaggle competition',
                                         tags = {'format':'CSV', 'purpose': 'training'},
                                         create_new_version = True)
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')
    
    
if 'titanic testing dataset' not in ws.datasets:
    default_ds.upload_files(files = ['./test.csv'],
                        target_path = 'titanic-data/',
                        overwrite = True,
                        show_progress = True
                       )
    
    try:
        tab_test_data = tab_test_data_set.register(workspace=ws,
                                         name = 'titanic testing dataset',
                                         description = 'titanic testing dataset for kaggle competition',
                                         tags = {'format':'CSV', 'purpose': 'testing'},
                                         create_new_version = True)
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')
    
    
    
#split dataset into training and validation subsets

titanic_train_ds = ws.datasets.get('titanic training dataset')
train_ds, test_ds = titanic_train_ds.random_split(percentage=0.7, seed = 123)
print("Data Ready!")   


Dataset already registered.
Dataset already registered.
Data Ready!


I'm going to try doing an auto run with auto featurization. I might try some other stuff after

In [5]:
from azureml.train.automl import AutoMLConfig


automl_config = AutoMLConfig(name='Automated ML Experiment',
                             task= 'classification',
                             compute_target='ms-learn-compute-ryan',
                             training_data = train_ds,
                             validation_data = test_ds,
                             label_column_name = 'Survived',
                             iterations = 10,
                             primary_metric = 'AUC_weighted',
                             max_concurrent_iterations=4,
                             featurization = 'auto'
                            )
print("ready for Auto ML run")

ready for Auto ML run


In [6]:
from azureml.core.experiment import Experiment
from azureml.widgets import RunDetails

print('Submitting Auto ML Experiment...')
automl_experiment = Experiment(ws, 'titanic_automl')
automl_run=automl_experiment.submit(automl_config)
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

Submitting Auto ML Experiment...
Running on remote or ADB.


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…


Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       DONE
DESCRIPTION:  If the missing values are expected, let the run complete. Otherwise cancel the current run and use a script to customize the handling of missing feature values that may be more appropriate based on the data type and business requirement.
              Learn more about missing value imputation: https://aka.ms/Automated

{'runId': 'AutoML_9ab411c0-9c93-4bfe-a0cf-22e3ed7880f6',
 'target': 'ms-learn-compute-ryan',
 'status': 'Completed',
 'startTimeUtc': '2020-08-19T20:24:59.027882Z',
 'endTimeUtc': '2020-08-19T20:28:19.715167Z',
 'properties': {'num_iterations': '10',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'ms-learn-compute-ryan',
  'AMLSettingsJsonString': '{"path":null,"name":"Automated ML Experiment","subscription_id":"375be063-544d-4b99-aed9-5a0e16b1f428","resource_group":"ms-learn-ml","workspace_name":"ms-learn-ml","region":"eastus","compute_target":"ms-learn-compute-ryan","spark_service":null,"azure_service":"remote","_local_managed_run_id":null,"iterations":10,"primary_metric":"AUC_weighted","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":null,"y_min":null,"y_max":null,"num_classes":null,"featuriz

In [7]:
best_run, fitted_model = automl_run.get_output()
print(best_run)
print(fitted_model)
best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

WARNING - The version of the SDK does not match the version the model was trained on.
WARNING - The consistency in the result may not be guaranteed.
WARNING - Package:azureml-automl-core, training version:1.11.0, current version:1.9.0.post1
Package:azureml-core, training version:1.11.0, current version:1.9.0
Package:azureml-dataprep, training version:2.0.2, current version:1.9.3
Package:azureml-pipeline-core, training version:1.11.0, current version:1.9.0
Package:azureml-telemetry, training version:1.11.0, current version:1.9.0
Package:azureml-train-automl-client, training version:1.11.0, current version:1.9.0.post1
WARNING - Below packages were used for model training but missing in current environment:
WARNING - Package:azureml-automl-runtime, training version:1.11.0
Package:azureml-dataset-runtime, training version:1.11.0.post1
Package:azureml-defaults, training version:1.11.0
Package:azureml-explain-model, training version:1.11.0
Package:azureml-interpret, training version:1.11.0
P

Run(Experiment: titanic_automl,
Id: AutoML_9ab411c0-9c93-4bfe-a0cf-22e3ed7880f6_3,
Type: azureml.scriptrun,
Status: Completed)
None
log_loss 0.4442005018712709
AUC_macro 0.8655005241090148
AUC_micro 0.8763091118800461
precision_score_weighted 0.8325974678915855
precision_score_macro 0.8197358197358198
norm_macro_recall 0.6470125786163523
f1_score_micro 0.8313725490196079
f1_score_weighted 0.8318666953961071
recall_score_weighted 0.8313725490196079
recall_score_macro 0.8235062893081762
average_precision_score_macro 0.836720891330272
f1_score_macro 0.8214896214896215
average_precision_score_micro 0.851149929780038
recall_score_micro 0.8313725490196079
AUC_weighted 0.8655005241090148
matthews_correlation 0.6432310583348505
accuracy 0.8313725490196079
precision_score_micro 0.8313725490196079
balanced_accuracy 0.8235062893081762
average_precision_score_weighted 0.850449445777353
weighted_accuracy 0.8383337681537525
confusion_matrix aml://artifactId/ExperimentRun/dcid.AutoML_9ab411c0-9c93-4b

In [8]:
print(best_run)

Run(Experiment: titanic_automl,
Id: AutoML_9ab411c0-9c93-4bfe-a0cf-22e3ed7880f6_3,
Type: azureml.scriptrun,
Status: Completed)


In [9]:
from azureml.core import Model

# Register model
best_run.register_model(model_path='outputs/model.pkl', model_name='titanic_model_automl',
                        tags={'Training context':'Auto ML'},
                        properties={'AUC': best_run_metrics['AUC_weighted'], 'Accuracy': best_run_metrics['accuracy']})

# List registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

titanic_model_automl version: 2
	 Training context : Auto ML
	 AUC : 0.8655005241090148
	 Accuracy : 0.8313725490196079


titanic_model_automl version: 1
	 Training context : Auto ML
	 AUC : 0.8692348008385744
	 Accuracy : 0.8627450980392157


diabetes_model version: 10
	 Training context : SKLearn Estimator (file dataset)
	 AUC : 0.846851712258014
	 Accuracy : 0.7788888888888889


diabetes_model version: 9
	 Training context : SKLearn Estimator (tabular dataset)
	 AUC : 0.8568613016622707
	 Accuracy : 0.7891111111111111


diabetes_model version: 8
	 Training context : Using Datastore
	 AUC : 0.846851712258014
	 Accuracy : 0.7788888888888889


diabetes_model version: 7
	 Training context : Parameterized SKLearn Estimator
	 AUC : 0.8483904671874223
	 Accuracy : 0.7736666666666666


diabetes_model version: 6
	 Training context : Estimator
	 AUC : 0.8484929598487486
	 Accuracy : 0.774


amlstudio-predict-diabetes version: 1
	 CreatedByAMLStudio : true


diabetes_model_automl version: 1
	 

In [10]:
titanic_testing_data = ws.datasets.get('titanic testing dataset')


In [11]:
titanic_testing_model = Model(ws, 'titanic_model_automl')

In [12]:
prediction = titanic_testing_model.predict(titanic_testing_data)

AttributeError: 'Model' object has no attribute 'predict'

In [13]:
print(titanic_testing_model.version)

2


In [61]:
type(titanic_testing_model)

azureml.core.model.Model

Now that I have the model I'm going to deploy it as a webservice than feed my data into it.

In [14]:
import os

folder_name = 'titanic_service'

#folder for the web service files
experiment_folder = './' + folder_name
os.makedirs(folder_name, exist_ok=True)

print(folder_name, 'folder created.')

titanic_service folder created.


In [15]:
%%writefile $folder_name/score_titanic.py
import json
import joblib
import numpy as np
from azureml.core.model import Model


# Called when service is loaded
def init():
    global model
    #ws=Workspace.get(name ='ms-learn-ml', subscription_id='375be063-544d-4b99-aed9-5a0e16b1f428', resource_group='ms-learn-ml')
    #get path to the deployed
    model_path = Model.get_model_path('titanic_model_automl')
    model = joblib.load(model_path)
    
#called when a request is received
def run(raw_data):
    
    #Get the input data as a numpy array
    data = np.array(json.loads(raw_data)['data'])
    #get a predication from the model
    predictions = model.predict(data)
    #get the corresponding classname for each predication (0 or 1)
    classnames = ["died", "survived"]
    predicted_classes = []
    for prediction in predictions:
        predicted_classes.append(classnames[prediction])
        
    #Return the predictions as JSON
    return json.dumps(predicted_classes)

Overwriting titanic_service/score_titanic.py


In [16]:
from azureml.core.conda_dependencies import CondaDependencies 

# Add the dependencies for our model (AzureML defaults is already included)
myenv = CondaDependencies()
myenv.add_conda_package("scikit-learn")

# Save the environment config as a .yml file
env_file = folder_name + "/titanic_env.yml"
with open(env_file,"w") as f:
    f.write(myenv.serialize_to_string())
print("Saved dependency info in", env_file)

# Print the .yml file
with open(env_file,"r") as f:
    print(f.read())

Saved dependency info in titanic_service/titanic_env.yml
# Conda environment specification. The dependencies defined in this file will

# be automatically provisioned for runs with userManagedDependencies=False.


# Details about the Conda environment file format:

# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually


name: project_environment
dependencies:
  # The python interpreter version.

  # Currently Azure ML only supports 3.5.2 and later.

- python=3.6.2

- pip:
    # Required packages for AzureML execution, history, and data preparation.

  - azureml-defaults

- scikit-learn
channels:
- anaconda
- conda-forge



In [18]:
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig

#configure the scoring environment
inference_config = InferenceConfig(runtime = "python",
                                   source_directory=folder_name,
                                   entry_script = "score_titanic.py",
                                   conda_file='titanic_env.yml')

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

service_name="titanic-service4"

service = Model.deploy(ws, service_name, [model], inference_config, deployment_config)

service.wait_for_deployment(True)
print(service.state)

Running..............................
Failed


ERROR - Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: f3461e50-06da-467e-9c1e-d77dec64618f
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs for your container instance: titanic-service3. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs. \nYou can also try to run image b1533106f06a49c8a9a7078ce1e0e1a3.azurecr.io/azureml/azureml_b4b48003799b80f7b408c8969bc8a0fd locally. Please refer to https://aka.ms/debugimage#service-launch-fails for more information.",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs fo

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: f3461e50-06da-467e-9c1e-d77dec64618f
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs for your container instance: titanic-service3. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs. \nYou can also try to run image b1533106f06a49c8a9a7078ce1e0e1a3.azurecr.io/azureml/azureml_b4b48003799b80f7b408c8969bc8a0fd locally. Please refer to https://aka.ms/debugimage#service-launch-fails for more information.",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Your container application crashed. This may be caused by errors in your scoring file'

In [55]:
service.get_logs()

'2020-08-19T18:58:16,099919506+00:00 - rsyslog/run \n2020-08-19T18:58:16,100058112+00:00 - gunicorn/run \n2020-08-19T18:58:16,104731995+00:00 - iot-server/run \n2020-08-19T18:58:16,104642992+00:00 - nginx/run \n/usr/sbin/nginx: /azureml-envs/azureml_4b824bcb98517d791c41923f24d65461/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_4b824bcb98517d791c41923f24d65461/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_4b824bcb98517d791c41923f24d65461/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_4b824bcb98517d791c41923f24d65461/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_4b824bcb98517d791c41923f24d65461/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)

In [72]:
print(folder_name)

titanic_service


In [83]:
from azureml.core.model import Model


print(Model.get_model_path('titanic_model_automl', version = 1))

azureml-models\titanic_model_automl\1\model.pkl
